In [1]:
import tensorflow as tf

def select_gpu(N):
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(gpus)
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus[N], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)    

select_gpu(2)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
3 Physical GPUs, 1 Logical GPUs


In [2]:
import re

In [3]:
sequence_length = 128
epochs = 10
model_name = 'roberta_zh_l12'

In [4]:
def txt_to_list(path, sequence_length):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
#         text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
#                 print(entity)            
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
        
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
#             while len(sentence) < sequence_length:
#                 sentence += 'O'
#             if len(sentence) > sequence_length:
#                 sentence = sentence[:sequence_length]
            x.append([i for i in sentence])
            
        tmp = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
#                 while len(tmp) < sequence_length:
#                     tmp.append('O')
#                 if len(tmp) > sequence_length:
#                     tmp = tmp[:sequence_length]
                y.append(tmp)
                tmp = list()
            else:
                tmp.append(i)
        y.append(label_list)
    return [x, y]

In [5]:
x1, y1 = txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt', sequence_length)

In [6]:
x2, y2 = txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt', sequence_length)

In [7]:
x4, y4 = txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt', sequence_length)

In [8]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76508
76508


In [9]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

48964 48964
12242 12242
15302 15302


In [10]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BertEmbedding
kashgari.config.use_cudnn_cell = True
embed_path = '/home/Danny/pretrain_model/{}'.format(model_name)
bert_embed = BertEmbedding(embed_path, sequence_length=sequence_length)
model = BiLSTM_CRF_Model(bert_embed)
history = model.fit(train_x,
                    train_y,
                    valid_x,
                    valid_y,
                    epochs=epochs,
                    batch_size=256,
                   )

2020-12-02 23:35:00,764 [DEBUG] kashgari - ------------------------------------------------
2020-12-02 23:35:00,765 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-02 23:35:00,765 [DEBUG] kashgari - config_path       : /home/Danny/pretrain_model/roberta_zh_l12/bert_config.json
2020-12-02 23:35:00,765 [DEBUG] kashgari - vocab_path      : /home/Danny/pretrain_model/roberta_zh_l12/vocab.txt
2020-12-02 23:35:00,766 [DEBUG] kashgari - checkpoint_path : /home/Danny/pretrain_model/roberta_zh_l12/bert_model.ckpt
2020-12-02 23:35:00,766 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30

Epoch 1/10
191/191 [==============================] - 43s 226ms/step - loss: 1.0126 - accuracy: 0.9520 - val_loss: 23.8103 - val_accuracy: 0.9829
Epoch 2/10
191/191 [==============================] - 40s 209ms/step - loss: 0.2734 - accuracy: 0.9831 - val_loss: 23.5657 - val_accuracy: 0.9868
Epoch 3/10
191/191 [==============================] - 40s 209ms/step - loss: 0.1989 - accuracy: 0.9866 - val_loss: 23.3627 - val_accuracy: 0.9889
Epoch 4/10
191/191 [==============================] - 40s 210ms/step - loss: 0.1642 - accuracy: 0.9882 - val_loss: 23.1667 - val_accuracy: 0.9895
Epoch 5/10
191/191 [==============================] - 40s 210ms/step - loss: 0.1434 - accuracy: 0.9891 - val_loss: 22.9730 - val_accuracy: 0.9893
Epoch 6/10
191/191 [==============================] - 40s 210ms/step - loss: 0.1258 - accuracy: 0.9900 - val_loss: 22.7818 - val_accuracy: 0.9897
Epoch 7/10
191/191 [==============================] - 40s 210ms/step - loss: 0.1141 - accuracy: 0.9902 - val_loss: 22.5781 -

In [11]:
model.evaluate(test_x, test_y)
model_path = 'model/{}-epoch-{}'.format(model_name, epochs)
model.save(model_path)

2020-12-02 23:42:02,231 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 72
2020-12-02 23:42:02,333 [DEBUG] kashgari - predict seq_length: None, input: (2, 15302, 72)


479/479 [==============================] - 44s 91ms/step


2020-12-02 23:42:50,032 [DEBUG] kashgari - predict output: (15302, 72)
2020-12-02 23:42:50,032 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]



                precision    recall  f1-score   support

      location     0.8247    0.8791    0.8511        91
          time     0.7860    0.7535    0.7694       848
          name     0.8864    0.7800    0.8298       100
    profession     0.0000    0.0000    0.0000         9
        family     0.8333    0.5882    0.6897        17
       contact     0.5556    0.3846    0.4545        13
      med_exam     0.7394    0.8714    0.8000       140
         money     0.6393    0.7500    0.6903        52
            ID     0.6667    0.2500    0.3636         8
clinical_event     0.0000    0.0000    0.0000         4
  organization     0.0000    0.0000    0.0000         3
     education     0.0000    0.0000    0.0000         1

     micro avg     0.7800    0.7582    0.7689      1286
     macro avg     0.7727    0.7582    0.7631      1286



2020-12-02 23:42:58,941 [INFO] kashgari - model saved to /home/Danny/AI-CUP-2020/Danny/kashgari/model/roberta_zh_l12-epoch-10


'/home/Danny/AI-CUP-2020/Danny/kashgari/model/roberta_zh_l12-epoch-10'

In [12]:
from matplotlib import pyplot as plt

In [13]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy', 'loss', 'val_loss'])
plt.show()

NameError: name 'history' is not defined

In [14]:
import pandas as pd
df = pd.read_csv('/home/Danny/AI-CUP-2020/datasets/stage4/development_2.csv')
df

,Unnamed: 0,article_id,text
0,0,0,醫師：這個要電腦看才有辦法，這是嚴重或一般的。民眾：這是沒關係啦。醫師：沒關係齁。民眾：他一...
1,1,1,醫師：就照舊這樣子。阿這一定要跟你講一下。家屬：沒關係啊。醫師：阿最近好嗎？民眾：都正常。醫...
2,2,2,醫師：就照這樣，我們就照這樣問診這樣而已啦，阿最近我們8月17號有抽血。家屬：對上上個禮拜。...
3,3,3,醫師：最近有怎麼樣嗎？民眾：都一樣欸。醫師：會痛還是怎麼樣嗎？民眾：都一樣。醫師：有發燒或是...
4,4,4,民眾：前一次是6月14號。醫師：嗯，14。民眾：13還是14。醫師：我看一下14啊，13和1...
...,...,...,...
65,65,65,個管師：來坐這裡。民眾：好，坐這裡嗎？好，靠你那麼近？個管師：靠我近一點好啊，不好嗎？會讓你...
66,66,66,個管師：好喘……哈哈哈哈。民眾：哈哈。個管師：好。小明這個月還好嗎？民眾：很好啊。個管師：很...
67,67,67,個管師：所以你是去篩檢的時候……民眾：恩。個管師：你去篩檢的時候然後許小姐跟你說你可以來這邊...
68,68,68,個管師：好喔，過去這一個月還好嗎？民眾：恩，還可以。個管師：那今天是請假？民眾：沒有，晚上上...


In [15]:
def predict_NER(model, text, sequence_length):
    x_list = list()
    while len(text) > sequence_length:
        x_list.append([i for i in text[:sequence_length]])
        text = text[sequence_length:]        
    x_list.append([i for i in text[:sequence_length]])
    y_list = model.predict(x_list)
    for i, j in zip(x_list, y_list):
        print(len(i), len(j))
    merge_list = list()
    for i in y_list:
        for j in i:
            merge_list.append(j)
    return merge_list

In [16]:
def output_NER(article_id, text, y):
    output = str()
    flag = False
    for i, j in enumerate(y):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            output += '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
    return output

In [17]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for article_id, text in zip(df['article_id'], df['text']):
    y = predict_NER(model, text, sequence_length)
    output += output_NER(article_id, text, y)

2020-12-02 23:59:15,169 [DEBUG] kashgari - predict seq_length: None, input: (2, 27, 130)


1/1 [==============================] - 0s 702us/step


2020-12-02 23:59:18,448 [DEBUG] kashgari - predict output: (27, 130)
2020-12-02 23:59:18,448 [DEBUG] kashgari - predict output argmax: [[ 0  1  1 ...  1  1  0]
 [ 0  1  1 ...  1  1  0]
 [ 0 10  7 ...  1  1  0]
 ...
 [ 0  1  1 ...  1  1  0]
 [ 0  1  1 ...  1  1  0]
 [ 0  1  1 ...  1  1  1]]
2020-12-02 23:59:18,451 [DEBUG] kashgari - predict seq_length: None, input: (2, 7, 130)


128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
45 45
1/1 [==============================] - 0s 762us/step


2020-12-02 23:59:18,552 [DEBUG] kashgari - predict output: (7, 130)
2020-12-02 23:59:18,555 [DEBUG] kashgari - predict output argmax: [[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1 10  7  7  1  1  1  1 10  7  7  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  0]
 [ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
   1  1  1  1  1  1  1  1  1  0]
 [ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  

...

128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
11 11
1/1 [==============================] - 0s 805us/step


2020-12-02 23:59:30,300 [DEBUG] kashgari - predict output: (6, 130)
2020-12-02 23:59:30,302 [DEBUG] kashgari - predict output argmax: [[0 1 1 1 1 1 1 1 1 1 6 4 4 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 6 4 1 1 1 1 1 1 1 6 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 6 4 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

128 128
128 128
128 128
128 128
128 128
107 107
1/1 [==============================] - 0s 955us/step


2020-12-02 23:59:30,432 [DEBUG] kashgari - predict output: (15, 130)
2020-12-02 23:59:30,433 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 ...
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 1]]


128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
128 128
99 99


In [20]:
print(output)

article_id	start_position	end_position	entity_text	entity_type
0	256	258	25	med_exam
0	273	276	李醫生	name
0	320	322	25	med_exam
0	326	328	25	med_exam
0	370	372	晚上	time
0	400	403	11點	time
0	615	619	11點多	time
0	644	647	這兩天	time
0	1168	1172	7月21	time
0	2200	2203	13號	time
0	2214	2217	15天	time
0	2226	2228	3天	time
0	2245	2247	三天	time
0	2319	2322	禮拜一	time
0	2332	2335	禮拜一	time
0	2372	2375	前幾天	time
0	2448	2451	這兩天	time
0	2555	2558	皇后區	location
0	2619	2622	下一趟	time
0	2624	2628	下下禮拜	time
0	2735	2740	布魯克林區	location
0	2752	2755	皇后區	location
0	2760	2763	皇后區	location
0	2807	2810	下禮拜	time
0	3029	3031	昨天	time
0	3067	3070	下禮拜	time
0	3339	3343	再下禮拜	time
1	59	62	108	med_exam
1	66	69	108	med_exam
1	457	460	五月中	time
1	671	674	三個月	time
2	27	32	8月17號	time
2	40	45	上上個禮拜	time
2	53	56	110	med_exam
2	85	88	110	med_exam
2	111	114	6.0	med_exam
2	253	256	126	med_exam
2	279	282	2年前	time
2	563	566	光華路	location
2	571	574	光華路	location
2	980	982	兩天	time
2	1096	1098	半年	time
2	1114	1116	半年	time
2	1118	1120	半年	time
2	1147	115

In [ ]:
output_path='output/{}_epo_{}.tsv'.format(model_name, epochs)
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output)